In [ ]:
!pip install pathway bokeh --quiet


**This notebook includes all three pricing models, real-time simulation using Pathway, and Bokeh visualizations, as required by the project guidelines.
**

In [ ]:
#  Library Imports for Dynamic Parking Pricing Project
# Core Libraries
import numpy as np # Numerical operations
import pandas as pd # Data manipulation and DataFrame operations
import matplotlib.pyplot as plt   # (Optional) Visualization support
# Time & Date Handling
from datetime import datetime
# Time & Date Handling
import pathway as pw   # Pathway library for real-time data simulation
# Visualization Libraries
import bokeh.plotting    # For interactive real-time plotting
import panel as pn
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, output_notebook, show
import time
#  Assumptions:
# - All features (queue length, traffic, vehicle weight) are pre-normalized [0,1]
# - Pricing starts at base price of $10
# - Timestamp ordering is maintained for all simulation
# - Real-time ingestion is simulated using Pathway on a static dataset


In [ ]:
#  Upload dataset manually in Google Colab environment
from google.colab import files
uploaded = files.upload()

In [ ]:
import pandas as pd
#loading the dataset
df = pd.read_csv('dataset.csv')

In [ ]:
#  Convert date and time columns into a single datetime object
df['timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')
#  Sort data by parking lot ID and timestamp to maintain chronological order
df.sort_values(by=['SystemCodeNumber', 'timestamp'], inplace=True)
#  Reset index after sorting for clean row indexing
df.reset_index(drop=True, inplace=True)

In [ ]:
# Feature Engineering
# Assign numeric weight to vehicle types (used in demand function)
vehicle_type_weight = {'car': 1.0, 'bike': 0.5}
df['vehicle_weight'] = df['VehicleType'].map(vehicle_type_weight)
# Encode traffic conditions into numeric levels (low = 1, high = 3)
traffic_encoding = {'low': 1, 'medium': 2, 'high': 3}
df['traffic_level'] = df['TrafficConditionNearby'].map(traffic_encoding)
# Calculate occupancy rate as a fraction of total capacity
df['occupancy_rate'] = df['Occupancy'] / df['Capacity']
# Normalize queue length to 0–1 scale
df['queue_norm'] = (df['QueueLength'] - df['QueueLength'].min()) / (df['QueueLength'].max() - df['QueueLength'].min())

# Normalize traffic level to range 0–1 (1 for low, 2 for medium, 3 for high)
df['traffic_norm'] = (df['traffic_level'] - 1) / 2

##  Model 1: Baseline Linear Pricing
This is a simple reference model to track how price changes with occupancy alone.
**Price = BasePrice + α × OccupancyRate**
Where:
- `BasePrice` = $10 (starting point)
- `α` = 2.5 (scaling factor)
- `OccupancyRate` = current occupancy / lot capacity
###  Behavior:
- As occupancy increases, price increases linearly.
- Price change is smooth and easy to interpret.
- No consideration of queue, traffic, vehicle type, or competitors.

This model serves as a benchmark for evaluating more advanced dynamic pricing models.

In [ ]:
# Model 1: Linear Pricing
base_price = 10
alpha = 2.5
df['price_model1'] = base_price + alpha * df['occupancy_rate']

##  Model 2: Demand-Based Dynamic Pricing

This model adjusts the parking price based on **real-time demand**, calculated using a weighted combination of relevant features.

**Demand Function:Demand = α·OccupancyRate + β·QueueNorm − γ·TrafficNorm + δ·IsSpecialDay + ε·VehicleWeight
**
Where:
- α = 0.6 → More occupancy → higher price
- β = 0.3 → Longer queue → higher price
- γ = 0.4 → More traffic → lower price
- δ = 0.2 → Special event/holiday → higher price
- ε = 0.5 → Cars (1.0) vs bikes (0.5)
We normalize this raw demand score and plug it into:
Price = BasePrice × (1 + λ × NormalizedDemand)
 λ = 1.2

###  Behavior:
- Smooth and explainable price changes.
- Responds to real-time conditions.
- More realistic than Model 1.

In [ ]:
# Model 2: Demand-Based Pricing
alpha_d, beta_d, gamma_d, delta_d, epsilon_d = 0.6, 0.3, 0.4, 0.2, 0.5
# Feature weights for demand function
# Raw demand score using a weighted combination of key factors
df['raw_demand'] = (
    alpha_d * df['occupancy_rate'] +  # More occupancy → higher demand
    beta_d * df['queue_norm'] -       # More queue → higher demand
    gamma_d * df['traffic_norm'] +    # More traffic → lower demand
    delta_d * df['IsSpecialDay'] +    # Special day (holiday/event) → higher demand
    epsilon_d * df['vehicle_weight']   # Larger vehicle → higher demand
)
df['norm_demand'] = (df['raw_demand'] - df['raw_demand'].min()) / (df['raw_demand'].max() - df['raw_demand'].min()) #normalization
lambda_price = 1.2
# Pricing factor to scale price change with demand
df['price_model2'] = base_price * (1 + lambda_price * df['norm_demand']) # Calculate dynamic price based on normalized demand
df['price_model2'] = df['price_model2'].clip(lower=5, upper=20) # Ensure prices stay within $5 to $20 range

##  Model 3: Competitive Pricing with Location Intelligence

This model builds on Model 2 by incorporating **competitor pricing** and **lot capacity pressure**.

**Logic Overview:**
1. For each timestamp, compute the **average price** of nearby parking lots (excluding self).
2. Check if the current lot is **overburdened** (occupancy > 95%).
3. Adjust the price based on competitor behavior:

If the parking lot is overburdened (occupancy > 95%) and nearby competitors have lower prices, then reduce the current lot's price by up to $2 (but not below $5).

If the nearby competitor lots are more expensive than the current lot, then slightly increase the current price by up to $1.5 (but not above $20).

If neither condition applies, the lot uses the same price as computed by Model 2 (demand-based pricing).

**Price boundaries:** All prices are kept between $5 and $20 for realism.

###  Strategic Behavior:
- Incentivizes rerouting when lots are overfull.
- Increases revenue potential when competitors are more expensive.
- Encourages price competition in high-demand areas.


In [ ]:
#  # Calculate nearby average competitor price (excluding self)
# Step 1: Calculate nearby average competitor price (excluding the current lot)
df['nearby_avg_price'] = df.groupby('timestamp')['price_model2'].transform(lambda x: (x.sum() - x) / (len(x) - 1))
# Step 2: Identify overburdened lots (95%+ occupancy)
# Check for overburdened lots
df['overburdened'] = df['occupancy_rate'] > 0.95

# Define Model 3 function
# Step 3: Define pricing logic based on competition and capacity
def compute_model3(row):
    base = row['price_model2']
    competitor_price = row['nearby_avg_price']
    overburdened = row['overburdened']
# Case 1: Lot is overburdened and competitors are cheaper → reduce price to attract fewer vehicles
    if overburdened and competitor_price < base:
        return max(base - 2, 5)
   # Case 2: Competitors are more expensive → raise price slightly
    elif competitor_price > base:
        return min(base + 1.5, 20)
  # Case 3: Default to Model 2 pricing
    else:
        return base

# Apply the logic
df['price_model3'] = df.apply(compute_model3, axis=1)
df['price_model3'] = df['price_model3'].round(2)


In [ ]:
import pathway as pw
from pathway.internals import dtype  # For DATE_TIME_UTC
import datetime

#  Define Pathway Schema
class RowSchema(pw.Schema):
    timestamp: dtype.DATE_TIME_UTC
    SystemCodeNumber: str
    occupancy_rate: float
    queue_norm: float
    traffic_norm: float
    IsSpecialDay: int
    vehicle_weight: float

#  UDF taking individual feature columns
@pw.udf
def calculate_price(occupancy_rate, queue_norm, traffic_norm, IsSpecialDay, vehicle_weight):
    demand = (
        0.6 * occupancy_rate +
        0.3 * queue_norm -
        0.4 * traffic_norm +
        0.2 * IsSpecialDay +
        0.5 * vehicle_weight
    )
    norm_demand = demand  # Already normalized to 0–1 scale
    price = 10 * (1 + 1.2 * norm_demand)
    return min(max(price, 5), 20)

#  Convert your dataframe to a Pathway table
table = pw.debug.table_from_pandas(
    df[['timestamp', 'SystemCodeNumber', 'occupancy_rate', 'queue_norm', 'traffic_norm', 'IsSpecialDay', 'vehicle_weight']],
    schema=RowSchema
)

# Use .select() to generate real-time pricing
result = table.select(
    timestamp=pw.this.timestamp,
    lot=pw.this.SystemCodeNumber,
    price=calculate_price(
        pw.this.occupancy_rate,
        pw.this.queue_norm,
        pw.this.traffic_norm,
        pw.this.IsSpecialDay,
        pw.this.vehicle_weight
    )
)

# Run Pathway pipeline
pw.run()





In [ ]:
# Real-Time Visualization using Bokeh
output_notebook() # Enable Bokeh to show plots in notebook
sample_lot = df[df['SystemCodeNumber'] == df['SystemCodeNumber'].iloc[0]] # Select a sample parking lot for visualization
source = ColumnDataSource(sample_lot) # Create a Bokeh data source from filtered data

p = figure(title="Real-Time Pricing (Model 2)", x_axis_type='datetime', width=700, height=300)  # Create a time-series line plot for Model 2 prices
p.line(x='timestamp', y='price_model2', source=source, legend_label='Model 2 Price', line_width=2, color='blue')# Plot Model 2 price over time
show(p)

In [ ]:
 # Ensure latest version of sample lot is used
sample_lot = df[df['SystemCodeNumber'] == df['SystemCodeNumber'].iloc[0]]

# Create Bokeh data source
source = ColumnDataSource(sample_lot)

# Enable inline plotting
output_notebook()

# Create the time-series plot
p = figure(title="Model Price Comparison (Sample Lot)", x_axis_type='datetime', width=750, height=350)

# Add lines for all three models
p.line(x='timestamp', y='price_model1', source=source, color='green', legend_label='Model 1', line_width=2)
p.line(x='timestamp', y='price_model2', source=source, color='blue', legend_label='Model 2', line_width=2)
p.line(x='timestamp', y='price_model3', source=source, color='red', legend_label='Model 3', line_width=2)

# Styling
p.legend.location = "top_left"
p.xaxis.axis_label = "Time"
p.yaxis.axis_label = "Price ($)"
p.title.text_font_size = '14pt'

# Display
show(p)



In [ ]:
 from bokeh.transform import dodge
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure, show, output_notebook

output_notebook()

# Take 1 row per unique SystemCodeNumber (to avoid duplicates in x_range)
sample = df.drop_duplicates(subset='SystemCodeNumber').sample(10, random_state=42)
bar_source = ColumnDataSource(sample)

# Ensure x_range has unique factors only
x_range_vals = sample['SystemCodeNumber'].astype(str).tolist()

# Create figure
bar = figure(x_range=x_range_vals, height=350, width=750,
             title="Model 3 vs Competitor Prices")

# Bar 1: Model 3
bar.vbar(x=dodge('SystemCodeNumber', -0.2, range=bar.x_range), top='price_model3',
         source=bar_source, width=0.4, legend_label='Model 3 Price', color='navy')

# Bar 2: Nearby Avg Price
bar.vbar(x=dodge('SystemCodeNumber', 0.2, range=bar.x_range), top='nearby_avg_price',
         source=bar_source, width=0.4, legend_label='Nearby Avg Price', color='orange')

# Style
bar.xaxis.axis_label = "Parking Lot"
bar.yaxis.axis_label = "Price ($)"
bar.legend.location = "top_left"
bar.legend.label_text_font_size = "10pt"
bar.title.text_font_size = "14pt"

show(bar)

